# Testing NN using MNIST dataset

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 9912422/9912422 [00:00<00:00, 17879934.58it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 2133421.87it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 10880338.62it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [13]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.881491  [   64/60000]
loss: 1.831724  [ 6464/60000]
loss: 1.945403  [12864/60000]
loss: 1.928551  [19264/60000]
loss: 1.792093  [25664/60000]
loss: 1.731927  [32064/60000]
loss: 1.771696  [38464/60000]
loss: 1.937162  [44864/60000]
loss: 1.828242  [51264/60000]
loss: 1.803262  [57664/60000]
Test Error: 
 Accuracy: 40.3%, Avg loss: 1.767705 

Epoch 2
-------------------------------
loss: 1.760217  [   64/60000]
loss: 1.714229  [ 6464/60000]
loss: 1.828657  [12864/60000]
loss: 1.780590  [19264/60000]
loss: 1.632468  [25664/60000]
loss: 1.591793  [32064/60000]
loss: 1.624379  [38464/60000]
loss: 1.804552  [44864/60000]
loss: 1.675171  [51264/60000]
loss: 1.631787  [57664/60000]
Test Error: 
 Accuracy: 51.1%, Avg loss: 1.611326 

Epoch 3
-------------------------------
loss: 1.617029  [   64/60000]
loss: 1.567302  [ 6464/60000]
loss: 1.666251  [12864/60000]
loss: 1.587654  [19264/60000]
loss: 1.450141  [25664/60000]
loss: 1.432996  [32064/600

In [18]:
model(X)

tensor([[-3.4700e-01, -3.6745e+00, -1.5450e+00,  1.2718e-01, -2.8728e+00,
          9.6658e-01, -3.2352e+00,  7.0909e+00,  2.5797e+00,  2.1125e+00],
        [ 5.0195e-01, -2.4320e+00,  6.0200e+00,  3.2430e+00, -4.7528e+00,
          2.2941e+00,  3.8646e+00, -5.1807e+00,  1.7404e+00, -5.6995e+00],
        [-5.4105e+00,  5.6299e+00,  1.5015e+00,  2.1017e+00, -2.3074e+00,
         -4.1601e-01, -1.1042e+00,  2.5610e-01,  2.2873e+00, -2.4298e-01],
        [ 8.9865e+00, -1.4127e+01,  4.8164e-01,  1.3206e+00, -4.1682e+00,
          4.4316e+00, -6.1623e-02,  1.4547e-01,  2.8883e+00, -2.2138e+00],
        [-5.7682e-01, -6.1576e+00, -7.3567e-02, -3.2124e+00,  4.5325e+00,
         -5.9435e-01,  3.6537e-01,  9.4135e-01,  4.2461e-01,  3.7683e+00],
        [-7.1220e+00,  7.5146e+00,  1.2673e+00,  3.2336e+00, -3.0438e+00,
         -8.0104e-01, -3.0128e+00,  1.3027e+00,  2.9972e+00,  7.2769e-01],
        [-3.0319e+00, -4.5748e+00, -5.0801e+00, -1.6865e+00,  5.2630e+00,
          2.1273e+00, -1.3166e+0